# EMA Crossover Backtest – Walkthrough

This notebook walks through a simple EMA crossover strategy on SPY.

We will:

1. Load SPY price data.
2. Compute fast and slow EMAs and plot them.
3. Build trading signals and positions.
4. Run a backtest with trading costs.
5. Look at equity, drawdown, and Sharpe ratio.
6. Compare long-only vs long/short and different EMA settings.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

from src.ema_backtest import (
    get_price_data,
    add_ema_signals,
    run_backtest,
    plot_price,
    plot_signals,
    plot_equity,
    plot_drawdown,
)

plt.rcParams["figure.figsize"] = (12, 6)

## 1. Load SPY data

We start by loading daily price data for SPY from 2015 onward and looking at the raw price series.

In [ ]:
df_raw = get_price_data("SPY", "2015-01-01")
df_raw.head()

In [ ]:
df_raw["price"].plot(title="SPY Adjusted Close")
plt.xlabel("Date")
plt.ylabel("Price")
plt.grid(True)
plt.show()

## 2. Add fast and slow EMAs

We now add a fast EMA and a slow EMA.

- Fast EMA reacts quickly to price moves.
- Slow EMA reacts more slowly and tracks the longer trend.

In [ ]:
FAST = 200
SLOW = 500

df = add_ema_signals(df_raw, fast=FAST, slow=SLOW, long_short=False)
plot_price(df)

The orange line (fast EMA) hugs the price more closely than the green line (slow EMA). 
Crossovers between these two lines will drive our trading signals.

## 3. Trading signal and position

The strategy is:

- Go long when the fast EMA is above the slow EMA.
- Stay flat otherwise (for long-only mode).

To avoid lookahead bias, we shift the signal by one day so trades happen on the next bar.

In [ ]:
df[["price", "ema_fast", "ema_slow", "signal", "position"]].head(15)

In [ ]:
plot_signals(df)

Green dots mark entries where we move from no position to long.
Red dots mark exits where we close the long position.

## 4. Run backtest with trading costs

We now run the backtest for this EMA pair with:

- 0.1% cost per unit of position change,
- 2% annual risk-free rate for the Sharpe ratio.

In [ ]:
COST_PER_TRADE = 0.001
ANNUAL_RF = 0.02

df_backtest, stats = run_backtest(
    df,
    cost_per_trade=COST_PER_TRADE,
    annual_rf=ANNUAL_RF,
)

stats

In [ ]:
plot_equity(df_backtest)
plot_drawdown(df_backtest)

- `total_return_strategy` is the net return from following the EMA crossover with costs.
- `total_return_buy_hold` is what you get from holding SPY over the same period.
- `max_drawdown` shows the worst peak-to-trough drop.
- `sharpe` is the risk-adjusted return.

We can now change parameters and compare.

## 5. Long-only vs long/short

We compare:

- Long-only EMA crossover (1 when fast > slow, 0 otherwise).
- Long/short EMA crossover (1 when fast > slow, -1 when fast < slow).

In [ ]:
def run_variant(fast, slow, long_short):
    df_var = add_ema_signals(df_raw, fast=fast, slow=slow, long_short=long_short)
    df_var, stats = run_backtest(
        df_var,
        cost_per_trade=COST_PER_TRADE,
        annual_rf=ANNUAL_RF,
    )
    return stats

pairs = [
    (12, 26),
    (50, 200),
    (500, 1000),
]

rows = []
for f, s in pairs:
    for long_short in [False, True]:
        stats = run_variant(f, s, long_short)
        rows.append({
            "fast": f,
            "slow": s,
            "long_short": long_short,
            **stats,
        })

results = pd.DataFrame(rows)
results

In [ ]:
results.sort_values("sharpe", ascending=False)

### Observations

- Long-only versions tend to do better on SPY because the index has an upward drift.
- Long/short versions suffer when we short during local drops that quickly reverse.
- Slower EMAs trade less, so trading costs are smaller.

## 6. What I learned

- How to compute and use EMAs for a crossover strategy.
- Why we must shift signals by one day to avoid lookahead bias.
- How to turn daily returns into an equity curve using cumulative products.
- How trading costs hit fast, flip-heavy strategies.
- Why long-only EMA on an index can behave very differently from long/short EMA.
- How to compare different parameter settings using a simple stats table.